# Demo2 : Schema Validation and Evolution in an S3 Datalake using Apache Hudi

## Table of Contents:

1. [Overview](#Overview)
2. [Bulk Insert the Initial Dataset](#Bulk-Insert-the-Initial-Dataset)
3. [Schema Validation](#Schema-Validation)
4. [Schema Evolution](#Schema-Evolution)

## Overview

This notebook is a continuation of the 1st notebook and covers the following concepts when writing Copy-On-Write tables to an S3 Datalake:

- Schema Validation.
- Schema Evolution.

There are 4 cases:

1. Columns are added in the middle - which is an incompatible change.
2. Columns are dropped i.e. missing - which is a compatible change as nulls are expected for newer records.
3. Column types are changed - is an **incompatible** change. You could try automatic casting as a resolution here.
4. Columns are added at the end - which is a compatible change.

The assumption is that all case mismatches in columns has been resolved by lower casing the fields. 

**This demo was run on a single node (r5.4xlarge) EMR Cluster version 5.29.**

Let's start by initializing the Spark Session to connect this notebook to our Spark EMR cluster:

Note that the files hudi-spark-bundle.jar and spark-avro.jar are copied into HDFS.

In [1]:
%%configure -f
{
    "conf":  { 
             "spark.jars":"hdfs:///hudi-spark-bundle.jar,hdfs:///spark-avro.jar",
             "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
             "spark.sql.hive.convertMetastoreParquet":"false",
             "spark.dynamicAllocation.executorIdleTimeout": 3600,
             "spark.executor.memory": "7G",
             "spark.executor.cores": 1,
             "spark.dynamicAllocation.initialExecutors":16
           } 
}

In [8]:
## CHANGE ME ##
config = {
    "table_name": "example_hudi_table",
    "target": "s3://neilawstmp2/tmp/hudi/example_hudi_table",
    "primary_key": "id",
    "sort_key": "sk",
    "commits_to_retain": "2"
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The constants for Python to use:

In [3]:
# General Constants
HUDI_FORMAT = "org.apache.hudi"
TABLE_NAME = "hoodie.table.name"
RECORDKEY_FIELD_OPT_KEY = "hoodie.datasource.write.recordkey.field"
PRECOMBINE_FIELD_OPT_KEY = "hoodie.datasource.write.precombine.field"
OPERATION_OPT_KEY = "hoodie.datasource.write.operation"
BULK_INSERT_OPERATION_OPT_VAL = "bulk_insert"
UPSERT_OPERATION_OPT_VAL = "upsert"
BULK_INSERT_PARALLELISM = "hoodie.bulkinsert.shuffle.parallelism"
UPSERT_PARALLELISM = "hoodie.upsert.shuffle.parallelism"
S3_CONSISTENCY_CHECK = "hoodie.consistency.check.enabled"
HUDI_CLEANER_POLICY = "hoodie.cleaner.policy"
KEEP_LATEST_COMMITS = "KEEP_LATEST_COMMITS"
HUDI_COMMITS_RETAINED = "hoodie.cleaner.commits.retained"
PAYLOAD_CLASS_OPT_KEY = "hoodie.datasource.write.payload.class"
EMPTY_PAYLOAD_CLASS_OPT_VAL = "org.apache.hudi.EmptyHoodieRecordPayload"

# Hive Constants
HIVE_SYNC_ENABLED_OPT_KEY="hoodie.datasource.hive_sync.enable"
HIVE_PARTITION_FIELDS_OPT_KEY="hoodie.datasource.hive_sync.partition_fields"
HIVE_ASSUME_DATE_PARTITION_OPT_KEY="hoodie.datasource.hive_sync.assume_date_partitioning"
HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY="hoodie.datasource.hive_sync.partition_extractor_class"
HIVE_TABLE_OPT_KEY="hoodie.datasource.hive_sync.table"

# Partition Constants
NONPARTITION_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.NonPartitionedExtractor"
MULIPART_KEYS_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.MultiPartKeysValueExtractor"
KEYGENERATOR_CLASS_OPT_KEY="hoodie.datasource.write.keygenerator.class"
NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.NonpartitionedKeyGenerator"
COMPLEX_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.ComplexKeyGenerator"
PARTITIONPATH_FIELD_OPT_KEY="hoodie.datasource.write.partitionpath.field"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Some helper functions:

In [5]:
## Generates Data
def get_new_json_data(start, count, increment=0):
    data = [{"id": i, "sk": i+increment, "txt": chr(65 + (i % 26))} for i in range(start, start + count)]
    return data

# Creates the Dataframe
def create_json_df(spark, data):
    sc = spark.sparkContext
    return spark.read.json(sc.parallelize(data, 2))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Bulk Insert the Initial Dataset

Let's generate 4M records to load into our Data Lake:

In [5]:
spark.sql("drop table "+config['table_name']).show(100,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

++
||
++
++

In [6]:
df1 = create_json_df(spark, get_new_json_data(0, 4000000))
print(df1.count())
df1.printSchema()
df1.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4000000
root
 |-- id: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: string (nullable = true)

+---+---+---+
| id| sk|txt|
+---+---+---+
|  0|  0|  A|
|  1|  1|  B|
|  2|  2|  C|
+---+---+---+
only showing top 3 rows

And write the data to S3:

In [7]:
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 3)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .mode("Overwrite")
      .save(config['target']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
spark.sql("show tables").show(100,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------------+-----------+
|database|tableName         |isTemporary|
+--------+------------------+-----------+
|default |example_hudi_table|false      |
+--------+------------------+-----------+

In [9]:
spark.sql("show create table "+config['table_name']).show(100,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                                                                                                                                          

## Schema Validation

Let's see how we can easily implement Schema Validation in Apache Hudi with just a few lines of code.

There are 4 cases:


1. Columns are added in the middle - which is an incompatible change.
2. Columns are dropped i.e. missing - which is a compatible change as nulls are expected for newer records.
3. Column types are changed - is an **incompatible** change. You could try automatic casting as a resolution here.
4. Columns are added at the end - which is a compatible change

### Adding Columns in the middle

Let's add a new column called 'new_col':

In [10]:
## Generates Data
def get_json_data(start, count, increment=0):
    data = [{"id": i, "sk": i+increment, "new_col": i, "txt": chr(65 + (i % 26))} for i in range(start, start + count)]
    return data

idf = create_json_df(spark, get_json_data(0, 100))
idf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- new_col: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: string (nullable = true)

In [78]:
def validateSchema(df, table):
    original_df=spark.sql("SELECT * FROM "+table+" LIMIT 0")
    columns_to_drop = ['_hoodie_commit_time', '_hoodie_commit_seqno','_hoodie_record_key','_hoodie_partition_path','_hoodie_file_name']
    odf = original_df.drop(*columns_to_drop)
    if (df.schema != odf.schema):
        print ("Schema Validation Failed : Incoming Schema is not compatible with existing Table.")
        if len(odf.schema) > len(df.schema):
            print ("Original Data Diff : "+str(set(odf.schema)-set(df.schema)))
        else:
            print ("Incoming Data Diff : "+str(set(df.schema)-set(odf.schema)))
        return False
    return True

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
validateSchema(idf,config['table_name'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Schema Validation Failed : Incoming Schema is not compatible with existing Table.
Incoming Data Diff : {StructField(new_col,LongType,true)}
False

Our validation function identified that a new column has been added:

Let's see what would happen if we try to insert this data:

In [12]:
(idf.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .mode("Append")
      .save(config['target']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o142.save.
: org.apache.hudi.hive.HoodieHiveSyncException: Failed in executing SQL ALTER TABLE `default.example_hudi_table` REPLACE COLUMNS(`_hoodie_commit_time` string, `_hoodie_commit_seqno` string, `_hoodie_record_key` string, `_hoodie_partition_path` string, `_hoodie_file_name` string, `id` bigint, `new_col` bigint, `sk` bigint, `txt` string )
	at org.apache.hudi.hive.HoodieHiveClient.updateHiveSQL(HoodieHiveClient.java:488)
	at org.apache.hudi.hive.HoodieHiveClient.updateTableDefinition(HoodieHiveClient.java:264)
	at org.apache.hudi.hive.HiveSyncTool.syncSchema(HiveSyncTool.java:148)
	at org.apache.hudi.hive.HiveSyncTool.syncHoodieTable(HiveSyncTool.java:95)
	at org.apache.hudi.hive.HiveSyncTool.syncHoodieTable(HiveSyncTool.java:67)
	at org.apache.hudi.HoodieSparkSqlWriter$.syncHive(HoodieSparkSqlWriter.scala:236)
	at org.apache.hudi.HoodieSparkSqlWriter$.write(HoodieSparkSqlWriter.scala:169)
	at org.apache.hudi.DefaultSour

We see that Hudi's Hive Sync step would have thrown the following error message:

org.apache.hive.service.cli.HiveSQLException: Error while processing statement: 
FAILED: Execution Error, return code 1 from org.apache.hadoop.hive.ql.exec.DDLTask. 
Unable to alter table. **The following columns have types incompatible with the existing columns in their respective positions :
sk**

### Adding Columns at the end

Let's recreate our table:

In [23]:
spark.sql("drop table "+config['table_name']).show(100,False)
spark.sql("show tables").show(100,False)
df1 = create_json_df(spark, get_new_json_data(0, 4000000))
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 3)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .mode("Overwrite")
      .save(config['target']))
spark.sql("show tables").show(100,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

++
||
++
++

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+

+--------+------------------+-----------+
|database|tableName         |isTemporary|
+--------+------------------+-----------+
|default |example_hudi_table|false      |
+--------+------------------+-----------+

In [25]:
## Generates Data
def get_json_data(start, count, increment=0):
    data = [{"id": i, "sk": i+increment, "new_col": i, "txt": chr(65 + (i % 26))} for i in range(start, start + count)]
    return data

idf = create_json_df(spark, get_json_data(0, 100)).select("id","sk","txt","new_col")
idf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: string (nullable = true)
 |-- new_col: long (nullable = true)

In [26]:
(idf.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .mode("Append")
      .save(config['target']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Note that we added new columns at the end, the Hive sync step did not complain as the schema is now backwards compatible.

Let's query our table to verify that the new_col has been added.

In [27]:
df=spark.sql("select * from "+config['table_name'])
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- id: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: string (nullable = true)
 |-- new_col: long (nullable = true)

### Changing Column Types

Let's change the type of one column this time:

In [44]:
spark.sql("drop table "+config['table_name']).show(100,False)
spark.sql("show tables").show(100,False)
df1 = create_json_df(spark, get_new_json_data(0, 4000000))
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 3)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .mode("Overwrite")
      .save(config['target']))
spark.sql("show tables").show(100,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

++
||
++
++

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+

+--------+------------------+-----------+
|database|tableName         |isTemporary|
+--------+------------------+-----------+
|default |example_hudi_table|false      |
+--------+------------------+-----------+

In [55]:
## Generates Data
def get_json_data(start, count, increment=0):
    data = [{"id": i, "sk": i+increment, "txt": (65 + (i % 26))} for i in range(start, start + count)]
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
df1 = create_json_df(spark, get_json_data(0, 100))
df1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: long (nullable = true)

In [57]:
df1.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+---+
| id| sk|txt|
+---+---+---+
|  0|  0| 65|
|  1|  1| 66|
|  2|  2| 67|
+---+---+---+
only showing top 3 rows

In [58]:
validateSchema(df1,config['table_name'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Schema Validation Failed : Incoming Schema is not compatible with existing Table.
Incoming Data Diff : {StructField(txt,LongType,true)}
False

Our function identifed that column types have changed.

In [59]:
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .mode("Append")
      .save(config['target']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o1030.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 287.0 failed 4 times, most recent failure: Lost task 2.3 in stage 287.0 (TID 1928, ip-172-31-37-168.us-west-2.compute.internal, executor 2): org.apache.hudi.exception.HoodieUpsertException: Error upserting bucketType UPDATE for partition :2
	at org.apache.hudi.table.HoodieCopyOnWriteTable.handleUpsertPartition(HoodieCopyOnWriteTable.java:264)
	at org.apache.hudi.HoodieWriteClient.lambda$upsertRecordsInternal$507693af$1(HoodieWriteClient.java:428)
	at org.apache.spark.api.java.JavaRDDLike$$anonfun$mapPartitionsWithIndex$1.apply(JavaRDDLike.scala:102)
	at org.apache.spark.api.java.JavaRDDLike$$anonfun$mapPartitionsWithIndex$1.apply(JavaRDDLike.scala:102)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsWithIndex$1$$anonfun$apply$25.apply(RDD.scala:853)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsWithIndex$1$$anonfun$apply$25.a

We see that Hudi has failed with the errors message when it tried to read the existing data as a number:
    
java.lang.UnsupportedOperationException: org.apache.parquet.avro.AvroConverters$FieldLongConverter

### Dropping Columns


In [60]:
spark.sql("drop table "+config['table_name']).show(100,False)
spark.sql("show tables").show(100,False)
df1 = create_json_df(spark, get_new_json_data(0, 4000000))
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 3)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .mode("Overwrite")
      .save(config['target']))
spark.sql("show tables").show(100,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

++
||
++
++

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+

+--------+------------------+-----------+
|database|tableName         |isTemporary|
+--------+------------------+-----------+
|default |example_hudi_table|false      |
+--------+------------------+-----------+

In [71]:
## Generates Data
def get_json_data(start, count, increment=0):
    data = [{"id": i, "sk": i+increment} for i in range(start, start + count)]
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
df1 = create_json_df(spark, get_json_data(0, 100))
df1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- sk: long (nullable = true)

In [73]:
df1.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+
| id| sk|
+---+---+
|  0|  0|
|  1|  1|
|  2|  2|
+---+---+
only showing top 3 rows

In [77]:
validateSchema(df1,config['table_name'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Schema Validation Failed : Incoming Schema is not compatible with existing Table.
Original Data Diff : {StructField(txt,StringType,true)}
False

In [64]:
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .mode("Append")
      .save(config['target']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o1144.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 320.0 failed 4 times, most recent failure: Lost task 1.3 in stage 320.0 (TID 2149, ip-172-31-37-168.us-west-2.compute.internal, executor 3): org.apache.hudi.exception.HoodieUpsertException: Error upserting bucketType UPDATE for partition :1
	at org.apache.hudi.table.HoodieCopyOnWriteTable.handleUpsertPartition(HoodieCopyOnWriteTable.java:264)
	at org.apache.hudi.HoodieWriteClient.lambda$upsertRecordsInternal$507693af$1(HoodieWriteClient.java:428)
	at org.apache.spark.api.java.JavaRDDLike$$anonfun$mapPartitionsWithIndex$1.apply(JavaRDDLike.scala:102)
	at org.apache.spark.api.java.JavaRDDLike$$anonfun$mapPartitionsWithIndex$1.apply(JavaRDDLike.scala:102)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsWithIndex$1$$anonfun$apply$25.apply(RDD.scala:853)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsWithIndex$1$$anonfun$apply$25.a

Hudi threw an error message:
    
Caused by: org.apache.parquet.io.InvalidRecordException: Parquet/Avro schema mismatch: Avro field 'txt' not found

What if we added a null column called 'txt'.

In [65]:
from pyspark.sql.functions import when, lit, col
from pyspark.sql.types import *

df1 = create_json_df(spark, get_json_data(0, 100)).withColumn("txt",lit(None).cast(StringType()))
df1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: string (nullable = true)

In [66]:
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .mode("Append")
      .save(config['target']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
spark.sql("select id,sk, txt from "+config['table_name']+' where id in (0,10000)').show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+----+
|   id|   sk| txt|
+-----+-----+----+
|    0|    0|null|
|10000|10000|   Q|
+-----+-----+----+

## Schema Evolution

To summarize: we tested the 4 cases:

1. Columns are added at the end - which is a compatible change
2. Columns are added in the middle - which is an incompatible change but could be resolved by rearranging the input columns.
3. Columns are dropped - which is an incompatible change but could be resolved the adding a null column. 
4. Column types are changed - which is an incompatible change and can potentially be resolved by casting the changed column to the right type.

Let's add columns at the end to ensure our schema remains compatible with the original schema.

In [6]:
## Generates Data
def get_json_data(start, count, increment=0):
    data = [{"id": i, "sk": i+increment, "new_col": i, "txt": chr(65 + (i % 26))} for i in range(start, start + count)]
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
spark.sql("drop table if exists "+config['table_name']).show(100,False)
spark.sql("show tables").show(100,False)
df1 = create_json_df(spark, get_new_json_data(0, 4000000))
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 3)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .mode("Overwrite")
      .save(config['target']))
spark.sql("show tables").show(100,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+

+--------+------------------+-----------+
|database|tableName         |isTemporary|
+--------+------------------+-----------+
|default |example_hudi_table|false      |
+--------+------------------+-----------+

In [70]:
def evolveSchema(df, table,strict=False,forcecast=True):
    original_df=spark.sql("SELECT * FROM "+table+" LIMIT 0")
    columns_to_drop = ['_hoodie_commit_time', '_hoodie_commit_seqno','_hoodie_record_key','_hoodie_partition_path','_hoodie_file_name']
    odf = original_df.drop(*columns_to_drop)
    if (df.schema != odf.schema):
        if (strict):
            print ("Strict Schema Validation Failed : Incoming Schema is not compatible with existing Table.")
            if len(odf.schema) > len(df.schema):
                print ("Original Data Diff : "+str(set(odf.schema)-set(df.schema)))
            else:
                print ("Incoming Data Diff : "+str(set(df.schema)-set(odf.schema)))
            return (False,df)
        else:
            new_cols=[s for s in list(set([s.name for s in df.schema.fields])-set([s.name for s in odf.schema.fields]))]
            if forcecast:
                ## force cast columns 
                existing_cols=[f'cast({s.name} as {s.dataType.typeName()}) {s.name}' for s in odf.schema.fields]
                #print  (existing_cols)
                #print  (new_cols)
                new_df = df.selectExpr(existing_cols+new_cols)
            else:    
                existing_cols=[s.name for s in odf.schema.fields]
                new_df = df.select(existing_cols+new_cols)
            ## re-arrange the columns to ensure schema compatibility
            
            return (True, new_df)
    return (True,df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We now perform a type change as well as add a new column.

In [77]:
## Generates Data
def get_json_data(start, count, increment=0):
    data = [{"id": i, "sk": i+increment, 'new':i, "txt": (65 + (i % 26))} for i in range(start, start + count)]
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
idf = create_json_df(spark, get_json_data(0, 100))
idf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- new: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: long (nullable = true)

In [79]:
(schemaValidationResult,evolvedDF)=evolveSchema(idf,config['table_name'],False)
evolvedDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: string (nullable = true)
 |-- new: long (nullable = true)

**We can now observe schema evolution has rearranged the columns to what is expected by our Hudi table and has fixed the datatypes as well.**

In [82]:
(evolvedDF.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .mode("Append")
      .save(config['target']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
df=spark.sql("select * from "+config['table_name'])
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- id: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: string (nullable = true)
 |-- new: long (nullable = true)